# Oficina 1: Modelo de previsão de temperaturas

# **Sumário**

<a id = ''> </a>

# **Entendimento do Problema:**

Você trabalha como cientista de dados em uma empresa de meteorologia que coleta dados diários de temperatura ao longo dos anos. O objetivo da empresa é prever a temperatura de amanhã com base nas temperaturas anteriores, para fornecer uma previsão mais precisa. Para isso, você decide usar uma rede neural do tipo LSTM (Long Short-Term Memory), que é adequada para lidar com dados sequenciais.

Seu objetivo nesta oficina é construir um modelo de deep learning utilizando uma rede neural LSTM que possa prever a temperatura do dia seguinte com base nas temperaturas dos últimos 30 dias.


# **Passo 1 - Preparação do Ambiente - Imports**

In [2]:
import numpy as np

import pandas as pd

# Visualização de dados e métricas de desempenho
import matplotlib.pyplot as plt

# 
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dense

from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'tensorflow.python'


# **Passo 1.1 - Estilo da Página**

<a id = ''> </a>

# **Passo 2: Carregamento e Preparação dos Dados**

Para simplificar, você pode usar um conjunto de dados fictício ou dados históricos reais de temperaturas médias diárias. Vamos criar um conjunto de dados fictício de temperaturas para os últimos 100 dias.

<a id = ''> </a>

# Passo 2.1: Gerando os dados


Código:

$$ y = \sin(\text{linspace}(0, 20, 100)) \cdot 10 + 25 + np.random.normal(0, 1, 100) $$

Equação: 

$$
y = 10 \cdot \sin(x) + 25 + \varepsilon
$$

Onde:

- ( x) é um conjunto de 100 valores igualmente espaçados no intervalo ([0,20]), ou seja,  
x \in \text{linspace}(0,20,100)

- $( \sin(x) $) representa a função seno aplicada a esses valores.  
- O termo $( 10 \cdot \sin(x) $) escala a amplitude da função seno.  
- O valor $( 25 $) é um deslocamento vertical.  
- $( \varepsilon \sim \mathcal{N}(0,1) $) representa um ruído aleatório seguindo uma distribuição normal com média 0 e desvio padrão 1.


$$
\textbf{A equação dada é:}
$$
$$
y = 10 \cdot \sin(x) + 25 + \varepsilon
$$

**Onde:**  
- $x$ varia de $0$ a $20$ (100 valores espaçados uniformemente).  
- $\sin(x)$ oscila entre $[-1,1]$, então $10 \cdot \sin(x)$ oscila entre $[-10,10]$.  
- O deslocamento $25$ eleva os valores para a faixa $[15,35]$ antes de adicionar o ruído.  
- $\varepsilon \sim \mathcal{N}(0,1)$, ou seja, é um ruído gaussiano com média 0 e desvio padrão 1.  

$$
\textbf{Estimativa dos valores extremos de } y
$$

**Sem o ruído ($\varepsilon = 0$):**  
- O valor mínimo ocorre em $\sin(x) = -1 \Rightarrow 10 \cdot (-1) + 25 = 15$.  
- O valor máximo ocorre em $\sin(x) = 1 \Rightarrow 10 \cdot (1) + 25 = 35$.  

**Com o ruído $\varepsilon \sim \mathcal{N}(0,1)$:**  
- Como $\varepsilon$ segue uma distribuição normal, ele geralmente está entre $[-3,3]$ (99,7% dos casos).  
- Assim, os valores de $y$ estarão aproximadamente entre:  
  $$
  15 - 3 = 12 \quad \text{(mínimo provável)}
  $$  
  $$
  35 + 3 = 38 \quad \text{(máximo provável)}
  $$  


In [2]:
# Gerando dados fictícios de temperaturas diárias para os últimos 100 dias

# A temperatura segue uma onda senoidal com um pouco de ruído para simular variação real

# Para garantir o mesmo resultado na geração dos dados sempre que o código for compilado é necessário garantir uma semente aleatório fixa.
np.random.seed(42)

data = np.sin(np.linspace(0, 20, 100)) * 10 + 25 + np.random.normal(0, 1, 100)

In [5]:
print(data)

[25.49671415 26.86822426 29.57905466 32.21937093 31.99547224 33.23471868
 35.94284007 35.64498165 34.52023733 35.23811954 33.54363677 32.48647082
 31.81586474 28.01494018 26.35717234 25.54831285 23.08110741 22.42837675
 19.34367482 17.17009492 18.63977374 15.85838113 15.42435704 13.59697403
 14.54808485 15.67711001 15.27262031 18.00557044 18.53526149 20.58874794
 22.19083364 26.64670222 26.7911721  27.68380138 31.34871962 30.86522433
 33.56660932 32.32981417 33.51520053 35.19378464 35.48231647 34.26582772
 32.95951676 31.42632134 28.62753479 27.55716288 25.85403112 25.35598262
 22.65518703 18.67058497 19.06549614 16.91867592 15.50221568 16.02496922
 16.06837309 15.99794969 14.66077185 16.01049593 17.82374893 19.94616508
 20.21481644 22.4068431  23.48259999 25.4058807  29.36260293 31.70991268
 31.86750522 34.24663622 34.57305107 34.15994608 35.36130541 36.32605853
 34.14219451 34.75936012 29.25835909 31.10363752 28.55758601 26.21917638
 24.59583938 20.52257545 20.39780949 19.26018355 18

In [8]:
# Verificando os valor máximo e mínimo do conjunto:
print(data.min())

print(data.max())

13.596974033994417
36.32605853415617


<a id = ''> </a>

# Passo 2.2: Preparação dos Dados

In [10]:
# Transformando os dados em um DataFrame para facilitar a manipulação:

df = pd.DataFrame(data, columns=['Temperatura'])

In [11]:
# Exibindo as primeiras 5 linhas:
df.head()

,Temperatura
0,25.496714
1,26.868224
2,29.579055
3,32.219371
4,31.995472


In [ ]:
# Visualizando os dados de temperatura para ter uma ideia das tendências:


# Passo 3: Pré-processamento dos Dados

# Passo 4: Construção da Rede LSTM

# Passo 5: Avaliação e Previsão